In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import sqrt
from random import randint

from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.layers import GRU
from keras.callbacks import EarlyStopping
from keras import initializers

from datetime import datetime
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Read Data
data = pd.read_csv('data/bitcoin2.csv')
data = data.dropna()
data.reset_index()

# Group Data
data['date'] = pd.to_datetime(data['Timestamp'],unit='s').dt.date
group = data.groupby('date')
Daily_Price = group['Weighted_Price'].mean()

# Split Test and Train
ntrain = int(len(Daily_Price) * 0.75)

df_train= Daily_Price[:ntrain - 1]
df_test= Daily_Price[ntrain:]

# Working Data
working_data = [df_train, df_test]
working_data = pd.concat(working_data)

working_data = working_data.reset_index()
working_data['date'] = pd.to_datetime(working_data['date'])
working_data = working_data.set_index('date')

In [3]:
# Look Back func
def create_lookback(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

In [4]:
from sklearn.preprocessing import MinMaxScaler

training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
test_set = df_test.values
test_set = np.reshape(test_set, (len(test_set), 1))

#scale datasets
scaler = MinMaxScaler()
training_set = scaler.fit_transform(training_set)
test_set = scaler.transform(test_set)

# create datasets which are suitable for time series forecasting
look_back = 1
X_train, Y_train = create_lookback(training_set, look_back)
X_test, Y_test = create_lookback(test_set, look_back)

 # reshape datasets so that they will be ok for the requirements of the LSTM model in Keras
X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
X_test = np.reshape(X_test, (len(X_test), 1, X_test.shape[1]))

In [5]:
# Load Model Func
def load_network(filename):
    #Load the h5 saved model and weights
    if(os.path.isfile(filename)):
        return load_model(filename)
    else:
        print('ERROR: "' + filename + '" file does not exist as a h5 model')
        return None

In [6]:
# add one additional data point to align shapes of the predictions and true labels
X_test = np.append(X_test, scaler.transform(working_data.iloc[-1][0]))
X_test = np.reshape(X_test, (len(X_test), -1, 1))

# Load model
model = load_network('data/line_model.h5')

# get predictions and then make some transformations to be able to calculate RMSE properly in USD
prediction = model.predict(X_test)
prediction_inverse = scaler.inverse_transform(prediction.reshape(-1, 1))
Y_test_inverse = scaler.inverse_transform(Y_test.reshape(-1, 1))
prediction2_inverse = np.array(prediction_inverse[:,0][1:])
Y_test2_inverse = np.array(Y_test_inverse[:,0])

In [7]:
# Chart Predictions
Test_Dates = Daily_Price[ntrain:].index

trace1 = go.Scatter(x=Test_Dates, y=Y_test2_inverse, name= 'Precio Actual',
                   line = dict(color = ('rgb(0, 96, 252)'),width = 3))
trace2 = go.Scatter(x=Test_Dates, y=prediction2_inverse, name= 'Precio Predicho',
                   line = dict(color = ('rgb(244, 146, 65)'),width = 4))
data = [trace1, trace2]
layout = dict(title = 'Comparación entre los valores predichos y los valores actuales.',
             xaxis = dict(title = 'Date'), yaxis = dict(title = 'Price, USD'))
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='results_demonstrating1')

In [8]:
# Mean Squared Error
RMSE = sqrt(mean_squared_error(Y_test2_inverse, prediction2_inverse))
print('Test RMSE: %.3f' % RMSE)

Test RMSE: 81.787


In [26]:
# Both Real and Prediction Data
date = datetime.strptime('2017-05-01', "%Y-%m-%d").date()
index_array = Daily_Price.index
almost_index = np.where(index_array.get_values() == date)
index = almost_index[0][0]
test_dates = Daily_Price[index:index+30]
test_dates = test_dates.reset_index()


date = datetime.strptime('2017-05-01', "%Y-%m-%d").date()
index_array = Daily_Price.index
almost_index = np.where(index_array.get_values() == date)
index = almost_index[0][0] - 1439
predicted_values = prediction2_inverse[index:index+30]

test_dates['predicted'] = predicted_values

frames = []
frames2 = []
for i, value in enumerate(predicted_values):
    if i > 1:   
        frames.append(
            {'data': [
                {
                    'x': frames[i-1]['data'][0]['x'] + [test_dates['date'][i]], 
                    'y': frames[i-1]['data'][0]['y'] + [predicted_values[i]],
                    'mode':'line',
                    'line': {
                        'simplify': False,
                        'color': ('rgb(244, 146, 65)'),
                        'width': 4
                    }
                }
            ]}
        )
        frames2.append(
            {'data': [
                {
                    'x': frames2[i-1]['data'][0]['x'] + [test_dates['date'][i]], 
                    'y': frames2[i-1]['data'][0]['y'] + [test_dates['Weighted_Price'][i]],
                    'mode':'line',
                    'line': {
                        'simplify': False,
                        'width': 4
                    }

                }
            ]}
        )
    else:
        frames.append(
            {'data': [
                {
                    'x': [test_dates['date'][i]], 
                    'y': [predicted_values[i]],
                    'mode':'line',
                    'line': {
                        'simplify': False,
                        'color': ('rgb(244, 146, 65)'),
                        'width': 4
                    }
                }
            ]}
        )
        frames2.append(
            {'data': [
                {
                    'x': [test_dates['date'][i]], 
                    'y': [test_dates['Weighted_Price'][i]],
                    'mode':'line',
                    'line': {
                        'simplify': False,
                        'width': 4
                    }
                }
            ]}
        )
        

figure1 = {'data': [{'x': [test_dates['date'][0]], 'y': [predicted_values[0]]}],
          'layout': {'xaxis': {'type': 'date', 'range': [test_dates['date'][0], test_dates['date'][-1:][29]] , 'autorange': False},
                     'yaxis': {'range': [predicted_values[0]-5, max(test_dates['Weighted_Price'])], 'autorange': False},
                     'title': 'Valores Predichos',
                     'updatemenus': [{'type': 'buttons',
                                      
                                     }]
                    },
          'frames': frames
         }

figure2 = {'data': [{'x': [test_dates['date'][0]], 'y': [test_dates['Weighted_Price'][0]]}],
          'layout': {'xaxis': {'type': 'date', 'range': [test_dates['date'][0], test_dates['date'][-1:][29]] , 'autorange': False},
                     'yaxis': {'range': [test_dates['Weighted_Price'][0]-5, max(test_dates['Weighted_Price'])], 'autorange': False},
                     'title': 'Valores Reales',
                     'updatemenus': [{'type': 'buttons',
                                      
                                     }]
                    },
          'frames': frames2
         }
import time
py.iplot(figure1)
time.sleep(2)
py.iplot(figure2)
max(test_dates['Weighted_Price'])


2515.7125148381965